In [1]:
# sys, file and nav packages:
import datetime as dt

# math packages:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF

# charting:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
import seaborn as sns

# home brew utitilties
import resources.utility_functions as ut
import resources.abundance_classes as ac
import resources.chart_kwargs as ck

import resources.sr_ut as sut

# images and display
import base64, io, IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display, Math, Latex


# set some parameters:
today = dt.datetime.now().date().strftime("%Y-%m-%d")
start_date = '2020-03-01'
end_date ='2021-05-31'

a_fail_rate = 50

unit_label = 'p/100m'
reporting_unit = 100

# name of the output folder:
name_of_project = 'check_data'

a_color = 'dodgerblue'

# colors for gradients
colors = ['beige', 'navajowhite', 'sandybrown', 'salmon', 'sienna']
nodes = [0.0, 0.2, 0.6, 0.8, 1.0]
cmap2 = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors)))

# set the maps
bassin_map = PILImage.open("resources/maps/survey_locations_all.jpeg")
bassin_pallette = {'rhone':'dimgray', 'aare':'salmon', 'linth':'tan', 'ticino':'steelblue', 'reuss':'purple'}

# define the feature level and components
comps = ['linth', 'rhone', 'aare', 'ticino']
comp_labels = {"linth":"Linth/Limmat", "rhone":"Rhône", 'aare':"Aare", "ticino":"Ticino/Cerisio", "reuss":"Reuss"}
comp_palette = {"Linth/Limmat":"dimgray", "Rhône":"tan", "Aare":"salmon", "Ticino/Cerisio":"steelblue", "Reuss":"purple"}

# explanatory variables:
luse_exp = ['% to buildings', '% to recreation', '% to agg', '% to woods', 'streets km', 'intersects']

# columns needed
use_these_cols = ['loc_date' ,
                  '% to buildings',
                  '% to trans', 
                  '% to recreation',
                  '% to agg',
                  '% to woods',
                  'population',
                  'river_bassin',
                  'water_name_slug',
                  'city',
                  'streets km',
                  'intersects',
                  'length',
                  'groupname',
                  'code'
                 ]

# these are default
top_name = ["All survey areas"]

# add the folder to the directory tree:
project_directory = ut.make_project_folder('output', name_of_project)

# get your data:
survey_data = pd.read_csv('resources/results_with_land_use_2015.csv')
river_bassins = ut.json_file_get("resources/river_basins.json")
dfBeaches = pd.read_csv("resources/beaches_with_land_use_rates.csv")
dfCodes = pd.read_csv("resources/codes_with_group_names_2015.csv")
dfDims = pd.read_csv("resources/dims_data.csv")

# set the index of the beach data to location slug
dfBeaches.set_index('slug', inplace=True)

city_map = dfBeaches['city']

# map locations to feature names
location_wname_key = dfBeaches.water_name_slug

# map water_name_slug to water_name
wname_wname = dfBeaches[['water_name_slug','water_name']].reset_index(drop=True).drop_duplicates()
wname_wname.set_index('water_name_slug', inplace=True)
        
def make_plot_with_spearmans(data, ax, n):
    """Gets Spearmans ranked correlation and make A/B scatter plot. Must proived a
    matplotlib axis object.
    """    
    sns.scatterplot(data=data, x=n, y=unit_label, ax=ax, color='black', s=30, edgecolor='white', alpha=0.6)
    corr, a_p = stats.spearmanr(data[n], data[unit_label])
    
    return ax, corr, a_p

dfCodes.set_index("code", inplace=True)

# these descriptions need to be shortened for display
dfCodes = sut.shorten_the_value(["G74", "description", "Insulation: includes spray foams"], dfCodes)
dfCodes = sut.shorten_the_value(["G940", "description", "Foamed EVA for crafts and sports"], dfCodes)
dfCodes = sut.shorten_the_value(["G96", "description", "Sanitary-pads/tampons, applicators"], dfCodes)
dfCodes = sut.shorten_the_value(["G178", "description", "Metal bottle caps and lids"], dfCodes)
dfCodes = sut.shorten_the_value(["G82", "description", "Expanded foams 2.5cm - 50cm"], dfCodes)
dfCodes = sut.shorten_the_value(["G81", "description", "Expanded foams .5cm - 2.5cm"], dfCodes)
dfCodes = sut.shorten_the_value(["G117", "description", "Expanded foams < 5mm"], dfCodes)
dfCodes = sut.shorten_the_value(["G75", "description", "Plastic/foamed polystyrene 0 - 2.5cm"], dfCodes)
dfCodes = sut.shorten_the_value(["G76", "description", "Plastic/foamed polystyrene 2.5cm - 50cm"], dfCodes)
dfCodes = sut.shorten_the_value(["G24", "description", "Plastic lid rings"], dfCodes)
dfCodes = sut.shorten_the_value(["G33", "description", "Lids for togo drinks plastic"], dfCodes)
dfCodes = sut.shorten_the_value(["G3", "description", "Plastic bags, carier bags"], dfCodes)
dfCodes = sut.shorten_the_value(["G204", "description", "Bricks, pipes not plastic"], dfCodes)

# make a map to the code descriptions
code_description_map = dfCodes.description

# make a map to the code descriptions
code_material_map = dfCodes.material

In [2]:
# define survey data pool
a_data = survey_data.copy()

# get a copy before any changes
a_dated = a_data[(a_data.date >= start_date)&(a_data.date <= end_date)].copy()

# format columns and slice by date
a_data['loc_date']=tuple(zip(a_data.location, a_data.date))
a_data['date']=pd.to_datetime(a_data['date'], format='%Y-%m-%d')
a_data = a_data[(a_data.date >= start_date)&(a_data.date <= end_date)]

# combine lugano and maggiore
a_data['river_bassin'] = a_data.river_bassin.where(a_data.river_bassin != 'tresa', 'ticino' )

# combine reuss and linth
a_data['river_bassin'] = a_data.river_bassin.where(a_data.river_bassin != 'reuss', 'linth' )

# convert meters of streets to kilometersf
a_data['streets'] = a_data.streets.astype('int')
a_data['streets km'] = a_data.streets/1000

# make a p/50m column, keep the pcs_m column
a_data[unit_label] = (a_data.pcs_m * reporting_unit).astype('int')

before_agg = a_data.copy()
# Combine the different sizes of fragmented plastics and styrofoam
# the codes for the foams, these will be called Gfoam
some_foams = ['G81', 'G82', 'G83']

# the codes for the fragmented plastics, these will be called Gfrags
some_frag_plas = list(a_data[a_data.groupname == 'plastic pieces'].code.unique())

# this extracts and aggregates the members of the two code groups, returns a dataframe
the_plast_rows = sut.create_aggregate_groups(a_data, codes_to_agg=some_frag_plas,a_model_code="G79", a_new_code="Gfrags")
the_foam_rows = sut.create_aggregate_groups(a_data, codes_to_agg=some_foams, a_model_code="G82", a_new_code="Gfoam")

# the foam codes and fragmented plastic codes have been aggregated in to Gfrags and Gfoam
a_data = sut.replace_a_group_of_codes_with_one(a_data, new_code_values=[the_plast_rows, the_foam_rows], codes_to_replace=[*some_frag_plas, *some_foams])
a_data['material'] = a_data.code.map(lambda x: code_material_map.loc[x])
a_data['city'] = a_data.location.map(lambda x: city_map.loc[x])

# survey totals
fd = a_data.copy()

# walensee landuse is approximated by comparing the land use profile from similar locations
# the classification for that part of switzerland is incomplete for the current estimates
# the previous one is 15 years old
# the land use profile of wychely - brienzersee was used for walenstadt-wyss (more prairie, buildings less woods)
# the land use profile of grand-clos - lac-leman was used for other locations on walensee (more woods, less buildings, less praire and agg)
luse_wstdt = dfBeaches.loc['wycheley'][['population','% to buildings', '% to trans', '% to recreation', '% to agg', '% to woods']]
estimate_luse = dfBeaches.loc['grand-clos'][['population','% to buildings', '% to trans', '% to recreation', '% to agg', '% to woods']]

# seperate out the locations that aren't walenstadt
wlsnsee_locs_not_wstdt = ['gasi-strand', 'untertenzen', 'mols-rocks', 'seeflechsen', 'seemuhlestrasse-strand', 'muhlehorn-dorf', 'murg-bad', 'flibach-river-right-bank']

for a_param in estimate_luse.index:
    fd.loc[fd.location.isin(wlsnsee_locs_not_wstdt), a_param] = estimate_luse[a_param]
    fd.loc[fd.location == 'walensee_walenstadt_wysse', a_param] = luse_wstdt[a_param]

# make aggregates and columns

# rivers and lakes
fd['w_t'] = 'r'
fd.loc[fd.water_name_slug.isin(dfBeaches[dfBeaches.water == 'l'].water_name_slug.unique()), 'w_t'] = 'l'

# month in string
fd['month'] = fd.date.dt.month
fd["eom"] = fd.date.map(lambda x: pd.Period(x,freq='M').end_time.date())

In [6]:
fd.quantity.sum() == a_dated.quantity.sum()

True

In [7]:
(fd.groupby('loc_date').quantity.sum().to_numpy() == a_dated.groupby('loc_date').quantity.sum().to_numpy()).any()

True

In [8]:
a_dated.groupby('loc_date').quantity.sum()

loc_date
('aabach', '2020-10-22')                                38
('aare-limmatspitz', '2020-07-13')                      70
('aare-port', '2021-04-23')                             99
('aare-solothurn-lido-strand', '2020-09-05')            27
('aare_bern_gerberm', '2020-09-23')                    134
                                                      ... 
('zurichsee_wollishofen_langendorfm', '2020-10-13')     65
('zurichsee_wollishofen_langendorfm', '2020-11-12')     41
('zurichsee_wollishofen_langendorfm', '2020-12-10')     50
('zurichsee_wollishofen_langendorfm', '2021-01-10')     34
('zurichsee_wollishofen_langendorfm', '2021-02-12')      8
Name: quantity, Length: 386, dtype: int64

In [10]:
a_dated[a_dated.duplicated() == True]

,date,code,pcs_m,quantity,location,loc_date,water_name_slug,river_bassin,groupname,population,...,intersects,string_date,fail,% to buildings,% to trans,% to recreation,% to agg,% to woods,% to water,% to unproductive


In [11]:
fd_locs = fd.location.unique()

# daily survey totals
dt_all = fd.groupby(['loc_date','location','river_bassin', 'water_name_slug','city','date'], as_index=False).agg({unit_label:'sum', 'quantity':'sum'})

# gather the municpalities and the population:
fd_pop_map = dfBeaches.loc[fd.location.unique()][['city','population']].copy()
fd_pop_map.drop_duplicates(inplace=True)
fd_pop_map.set_index('city', drop=True, inplace=True)

# gather the dimensional data for the time frame from dfDims
fd_dims= dfDims[(dfDims.location.isin(fd_locs))&(dfDims.date >= start_date)&(dfDims.date <= end_date)].copy()

# map the survey area name to the dims data record
m_ap_to_survey_area = fd[['location', 'river_bassin']].drop_duplicates().to_dict(orient='records')
a_new_map = {x['location']:x['river_bassin'] for x in m_ap_to_survey_area}
fd_dims['survey area'] = fd_dims.location.map(lambda x: ut.use_this_key(x, a_new_map))

# map length and area from dims to survey data
st_map = fd_dims[['loc_date', 'length', 'area']].to_dict(orient='records')
amap = {x['loc_date']:{'length':x['length'], 'area':x['area']}for x in st_map}
trans = {x:F"{x}"for x in fd.loc_date.unique()}

def this_map(x,amap,trans, var='length'):
    try:
        data = amap[trans[x]][var]
    except:
        data = 0
    return data  
    
fd['length'] = fd.loc_date.map(lambda x:  this_map(x,amap,trans, var='length'))
fd['area'] = fd.loc_date.map(lambda x:  this_map(x,amap,trans, var='area'))
fd['water'] = fd.location.map(lambda x: dfBeaches['water'][x])

# these surveys are missing area and length data. 
# use the average values from all the surveys at that location to fill in the missing values
make_lengths = fd.loc[fd.location.isin(['baby-plage-geneva','quai-maria-belgia'])].groupby('location').agg({'length':'mean', 'area':'mean'})
fd.loc[fd.loc_date == ('baby-plage-geneva', '2021-04-14'), 'length'] = 84
fd.loc[fd.loc_date == ('baby-plage-geneva', '2021-04-14'), 'area'] = 355
fd.loc[fd.loc_date.isin([('quai-maria-belgia', '2021-02-28'), ('quai-maria-belgia', '2021-01-31')]), 'length'] = 34
fd.loc[fd.loc_date.isin([('quai-maria-belgia', '2021-02-28'), ('quai-maria-belgia', '2021-01-31')]), 'area'] = 145

In [12]:
project_directory

'output/check_data'

In [13]:
fd.to_csv(F"{project_directory}/checked_sdata_eos_2020_21.csv", index=False)

In [14]:
before_agg.to_csv(F"{project_directory}/checked_before_agg_sdata_eos_2020_21.csv", index=False)

In [15]:
# collect the data before aggregating foams for all locations in the survye area
# group by loc_date and code

agg_pcs_quantity = {unit_label:'sum', 'quantity':'sum'}
# before_agg = a_dated.copy()
fd_frags_foams = before_agg[(before_agg.code.isin([*some_frag_plas, *some_foams]))&(before_agg.location.isin(fd_locs))].groupby(['loc_date','code'], as_index=False).agg(agg_pcs_quantity)
fd_frags_foams = fd_frags_foams.groupby('code').agg({unit_label:'mean', 'quantity':'sum'})

# add code description and format for printing
fd_frags_foams['item'] = fd_frags_foams.index.map(lambda x: code_description_map.loc[x])
fd_frags_foams['% of total'] = (fd_frags_foams.quantity/fd.quantity.sum()*100).round(2)
fd_frags_foams['% of total'] = fd_frags_foams['% of total'].map(lambda x: F"{x}%")
fd_frags_foams['quantity'] = fd_frags_foams['quantity'].map(lambda x: F"{x:,}")

# table data
data = fd_frags_foams[['item',unit_label, 'quantity', '% of total']]

In [16]:
# looking for conflict
# survey_data[survey_data.location.isin(['baby-plage-geneva', 'luscherz-plage'])][['date', 'loc_date', 'location']].drop_duplicates()

,date,loc_date,location
2192,2021-04-14,"('baby-plage-geneva', '2021-04-14')",baby-plage-geneva
2210,2021-03-10,"('baby-plage-geneva', '2021-03-10')",baby-plage-geneva
2227,2021-02-10,"('baby-plage-geneva', '2021-02-10')",baby-plage-geneva
2246,2021-01-16,"('baby-plage-geneva', '2021-01-16')",baby-plage-geneva
2281,2020-12-16,"('baby-plage-geneva', '2020-12-16')",baby-plage-geneva
2305,2020-11-18,"('baby-plage-geneva', '2020-11-18')",baby-plage-geneva
2323,2020-10-20,"('baby-plage-geneva', '2020-10-20')",baby-plage-geneva
2361,2020-08-18,"('baby-plage-geneva', '2020-08-18')",baby-plage-geneva
2385,2020-07-07,"('baby-plage-geneva', '2020-07-07')",baby-plage-geneva
2434,2020-06-09,"('baby-plage-geneva', '2020-06-09')",baby-plage-geneva


In [44]:
# # 
# was_lost = survey_data[(survey_data.location == 'baby-plage-geneva')&(survey_data.date == "2021-01-16")].copy()

# was_lost['nd'] = "2021-01-16"

# was_lost['loc_date'] = list(zip(was_lost.location, was_lost.date))

# was_lost['date'] = was_lost['nd']

# was_lost.drop('nd', axis=1, inplace=True)
# was_lost[was_lost.quantity > 0][['code', 'quantity']]


,code,quantity
2246,G204,1
2247,G78,6
2248,G24,3
2249,G904,2
2250,G928,2
2251,G29,1
2252,G95,1
2253,G134,1
2254,G33,4
2255,G159,1


In [38]:
# fd[(fd.location.isin(['luscherz-plage']))&(fd.date == '2020-12-26')&(fd.quantity > 0)][['code', 'quantity']]

,code,quantity
6840,G32,1
6841,G112,1
6842,G59,1
6843,G177,2
6844,G70,1
6845,G200,13
6846,G904,1
6847,G23,1
6848,G929,1
6849,G31,1


In [17]:
fd_dims= dfDims.copy()
# fd_dims.columns

In [18]:
# fd_dims[fd_dims.location.isin(['baby-plage-geneva'])]

,survey_key,date,length,area,mac_plast_w,mic_plas_w,total_w,est_weight,num_parts_staff,num_parts_other,time_minutes,participants,project,is_2020,location,loc_date,water_name_slug,river_bassin
13,baby-plage-geneva2020-05-05105,2020-05-05,105,630.0,250.0,0.000,0.450,0.0,1,3,150,"[""HD"",""ASL"",""EG""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2020-05-05')",lac-leman,rhone
14,baby-plage-geneva2020-06-09100,2020-06-09,100,550.0,690.0,1.265,0.691,0.0,1,3,35,"[""ASL"",""PRCS"",""HD""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2020-06-09')",lac-leman,rhone
15,baby-plage-geneva2020-07-07100,2020-07-07,100,400.0,350.0,16.115,0.830,0.0,1,10,90,"[""ASL"",""EG""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2020-07-07')",lac-leman,rhone
16,baby-plage-geneva2020-08-18100,2020-08-18,100,300.0,125.0,0.000,0.195,0.0,1,3,45,"[""ASL"",""EG""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2020-08-18')",lac-leman,rhone
17,baby-plage-geneva2020-10-2074,2020-10-20,74,370.0,130.0,0.240,0.165,0.0,1,0,180,"[""HD""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2020-10-20')",lac-leman,rhone
18,baby-plage-geneva2020-11-18100,2020-11-18,100,300.0,88.0,0.000,0.460,0.0,1,2,60,"[""ASL""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2020-11-18')",lac-leman,rhone
19,baby-plage-geneva2020-12-16100,2020-12-16,100,300.0,100.0,0.257,0.140,0.0,1,1,40,"[""ASL""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2020-12-16')",lac-leman,rhone
20,baby-plage-geneva2021-01-1653,2021-01-16,53,265.0,250.0,0.000,0.320,0.0,1,5,90,"[""HD"",""EG""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2021-01-16')",lac-leman,rhone
21,baby-plage-geneva2021-02-10100,2021-02-10,100,400.0,20.0,0.436,0.050,0.0,1,2,45,"[""ASL""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2021-02-10')",lac-leman,rhone
22,baby-plage-geneva2021-03-10100,2021-03-10,100,400.0,26.0,0.021,0.040,0.0,1,2,60,"[""ASL""]",2020,True,baby-plage-geneva,"('baby-plage-geneva', '2021-03-10')",lac-leman,rhone


In [41]:
# lp_dec_2020 =  fd_dims[fd_dims.location.isin(['luscherz-plage'])&(fd_dims.date == "2021-01-26")].copy()

# lp_dec_2020['survey_key'] = "luscherz-plage2020-12-2666"
# lp_dec_2020['date'] = "2020-12-26"
# lp_dec_2020['loc_date'] = list(zip(lp_dec_2020.location, lp_dec_2020.date))
# lp_dec_2020

,survey_key,date,length,area,mac_plast_w,mic_plas_w,total_w,est_weight,num_parts_staff,num_parts_other,time_minutes,participants,project,is_2020,location,loc_date,water_name_slug,river_bassin
160,luscherz-plage2020-12-2666,2020-12-26,66,132.0,70.0,0.013,0.15,0.0,1,0,120,"[""HD""]",2020,True,luscherz-plage,"(luscherz-plage, 2020-12-26)",bielersee,aare


In [20]:
fd_dims[fd_dims.survey_key.isin(["luscherz-plage2021-01-2666", "luscherz-plage2020-12-2666"])]


,survey_key,date,length,area,mac_plast_w,mic_plas_w,total_w,est_weight,num_parts_staff,num_parts_other,time_minutes,participants,project,is_2020,location,loc_date,water_name_slug,river_bassin
155,luscherz-plage2020-12-2666,2020-12-26,66,132.0,70.0,0.013,0.15,0.0,1,0,120,"[""HD""]",2020,True,luscherz-plage,"('luscherz-plage', '2020-12-26')",bielersee,aare


In [23]:


fd_dims.to_csv(F"{project_directory}/corrected_dims.csv")

In [51]:
fd_dims

,survey_key,date,length,area,mac_plast_w,mic_plas_w,total_w,est_weight,num_parts_staff,num_parts_other,time_minutes,participants,project,is_2020,location,loc_date,water_name_slug,river_bassin
0,aabach2020-10-2230,2020-10-22,30,75.0,75.0,0.145,0.080,0.0,1,0,90,"[""HD""]",2020,True,aabach,"('aabach', '2020-10-22')",zurichsee,linth
1,aare-limmatspitz2020-07-13120,2020-07-13,120,253.0,155.0,0.033,0.155,0.0,1,0,90,"[""HD""]",2020,True,aare-limmatspitz,"('aare-limmatspitz', '2020-07-13')",aare,aare
2,aare-port2021-04-2340,2021-04-23,40,411.0,130.0,0.000,0.180,0.0,1,0,145,"[""HD""]",2020,True,aare-port,"('aare-port', '2021-04-23')",aarenidau-buren-kanal,aare
3,aare-solothurn-lido-strand2020-09-0511,2020-09-05,11,38.0,1.0,0.000,0.001,0.0,1,0,80,"[""HD""]",2020,True,aare-solothurn-lido-strand,"('aare-solothurn-lido-strand', '2020-09-05')",aare,aare
4,aare_bern_gerberm2020-09-2337,2020-09-23,37,148.0,3800.0,0.000,5.300,0.0,1,1,150,"[""HD""]",2020,True,aare_bern_gerberm,"('aare_bern_gerberm', '2020-09-23')",aare,aare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,zurichsee_wollishofen_langendorfm2020-10-1366,2020-10-13,66,312.0,70.0,0.000,0.075,0.0,1,0,90,"[""HD""]",2020,True,zurichsee_wollishofen_langendorfm,"('zurichsee_wollishofen_langendorfm', '2020-10...",zurichsee,linth
393,zurichsee_wollishofen_langendorfm2020-11-1273,2020-11-12,73,363.0,50.0,0.000,0.260,0.0,2,0,120,"[""HD""]",2020,True,zurichsee_wollishofen_langendorfm,"('zurichsee_wollishofen_langendorfm', '2020-11...",zurichsee,linth
394,zurichsee_wollishofen_langendorfm2020-12-1068,2020-12-10,68,340.0,80.0,0.000,0.160,0.0,1,1,55,"[""HD""]",2020,True,zurichsee_wollishofen_langendorfm,"('zurichsee_wollishofen_langendorfm', '2020-12...",zurichsee,linth
395,zurichsee_wollishofen_langendorfm2021-01-1042,2021-01-10,42,231.0,30.0,0.000,0.210,0.0,1,1,50,"[""HD""]",2020,True,zurichsee_wollishofen_langendorfm,"('zurichsee_wollishofen_langendorfm', '2021-01...",zurichsee,linth
